
# AI 에이전트 개발 실습: Rule-based Planner + LLM Reasoner 하이브리드

이 노트북은 **LLM이 항상 구조화된 계획(JSON)을 안정적으로 생성하지 못한다**는 현실을 반영하여,  
교육/교재용으로 가장 안정적인 **하이브리드 에이전트 구조**를 구현합니다.

## 핵심 구조(역할 분리)
- **Planner(플래너)**: *무엇을/어떤 순서로/어떤 도구로* 할지 결정 (**규칙 기반**, 항상 성공)
- **Reasoner(리저너)**: 요약/정리/추론 등 **자연어 생성** 담당 (**LLM 기반**, 실패 가능 → fallback 설계)
- **Executor(실행기)**: 실제 도구 실행(계산, 파일 저장 등) (**코드 기반**, 항상 성공)

## 왜 하이브리드인가?
- LLM만으로 플래닝(JSON 계획 생성)을 맡기면 **빈 출력/형식 오류/스키마 불일치**가 자주 발생합니다.
- 본 실습은 플래닝을 **Rule-based(규칙)** 로 고정하고, LLM은 **두뇌(추론/요약)** 로만 사용합니다.
- 결과적으로 **‘안 멈추는’ 에이전트**를 만들면서, 에이전트 설계의 핵심인 **역할 분해**를 학습합니다.



## 0) 환경 준비 및 LLM(Reasoner) 로드

- 본 실습은 Hugging Face의 `google/flan-t5-base`(Text2Text) 모델을 사용합니다.
- *주의*: FLAN-T5는 **장문/엄격한 JSON 출력**에 약합니다. 따라서 **플래너는 규칙 기반**으로 설계합니다.


In [ ]:

!pip -q install transformers sentencepiece


In [ ]:

from transformers import pipeline
import os, json, re


In [ ]:

# -----------------------------
# LLM Reasoner 설정 (Flan-T5)
# -----------------------------
MODEL_NAME = "google/flan-t5-base"

llm = pipeline(
    "text2text-generation",
    model=MODEL_NAME,
    max_new_tokens=256,
    do_sample=False,
)

def llm_generate(prompt: str) -> str:
    '''LLM 호출 래퍼.
    - truncation=True로 긴 프롬프트에서도 오류를 줄입니다.
    - 빈 출력일 수 있으므로, 호출부에서는 fallback을 설계해야 합니다.
    '''
    out = llm(prompt, truncation=True)
    return (out[0].get("generated_text") or "").strip()

print("LLM ready:", MODEL_NAME)



## 1) Tools (Executor에서 호출)

에이전트가 “행동(Action)”을 하기 위해 사용하는 도구들입니다.

- `tool_calculator`: 사칙연산 계산(교육용으로 안전한 정규식 필터 포함)
- `tool_save_text`: 텍스트를 파일로 저장

> 실제 에이전트 시스템에서는 API 호출, DB 조회, 웹 검색 등이 여기에 해당합니다.


In [ ]:

def tool_calculator(expression: str) -> str:
    '''교육용 계산기 도구.
    - 허용 문자(숫자/연산자/괄호/공백)만 통과시켜 eval 위험을 줄입니다.
    '''
    allowed = re.compile(r"^[0-9\.\+\-\*\/\(\)\s]+$")
    if not allowed.match(expression):
        raise ValueError("허용되지 않은 수식입니다.")
    return str(eval(expression, {"__builtins__": {}}))

def tool_save_text(filepath: str, content: str) -> str:
    '''텍스트 파일 저장 도구.'''
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)
    return filepath



## 2) Reasoner: 회의록 요약 + To-do 생성

LLM(Reasoner)은 회의록을 읽고 요약/할 일을 생성합니다.  
하지만 LLM은 출력 형식을 지키지 못하거나 빈 출력이 나올 수 있습니다.  
따라서 **Rule-based Fallback**을 포함합니다.

### Fallback 전략
- LLM 출력이 불충분하면, 회의록 원문에서 **키워드 기반 핵심 문장**을 추출해 요약을 구성
- To-do도 빈약하면, 최소한의 의미 있는 To-do 템플릿으로 보완


In [ ]:

def llm_make_summary_todos(meeting_text: str):
    '''회의록 -> (요약 4줄, To-do 3개) 생성.
    1) LLM 시도
    2) 실패 시: Rule-based fallback으로 요약/To-do 보완
    '''
    meeting_text = (meeting_text or "").strip()

    # ===== 1) LLM 시도 =====
    prompt = f"""회의록을 한국어로 처리하라.
규칙:
- 요약은 4줄 이내(줄바꿈 유지)
- 할 일은 '-' bullet 3개
- 다른 문장 금지

회의록:
{meeting_text}
"""
    raw = llm_generate(prompt)

    lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]
    todos = [ln for ln in lines if ln.startswith("-")]
    summary_lines = [ln for ln in lines if not ln.startswith("-")]

    summary = "\n".join(summary_lines[:4]).strip()

    # LLM이 형식을 지켰다면 그대로 반환
    if summary and len(todos) >= 3:
        return summary, "\n".join(todos[:3])

    # ===== 2) Rule-based Fallback 요약 =====
    keywords = ["결정", "필요", "예산", "일정", "강화", "다음", "확정", "검토", "공유", "집행"]
    src_lines = [ln.strip("-• \t") for ln in meeting_text.splitlines() if ln.strip()]

    scored = []
    for ln in src_lines:
        score = sum(k in ln for k in keywords)
        if len(ln) >= 8:
            scored.append((score, ln))

    scored.sort(key=lambda x: (-x[0], -len(x[1])))
    summary_fb = "\n".join([ln for _, ln in scored[:4]])

    if not summary_fb:
        summary_fb = "\n".join(src_lines[:4]) if src_lines else "(요약 실패: 원문 없음)"

    # ===== 3) Rule-based To-do 생성 =====
    todos_fb = [
        "- 캠페인 일정 초안 정리 및 공유",
        "- 예산 집행 계획(2분기 기준) 수립",
        "- SNS 홍보 강화 방안 검토 및 제안",
    ]

    return summary_fb, "\n".join(todos_fb)



## 3) Rule-based Planner: 계획(steps) 생성

플래너는 **사용자 요청을 분석하여 실행 단계(steps)를 구성**합니다.  
여기서는 교육용으로 단순한 규칙을 사용합니다.

- 요청에 “회의/회의록”이 포함되면: 요약/To-do 생성 + 계산 + 파일 저장
- 그 외: 파일 저장만 수행


In [ ]:

def classify_request(user_request: str) -> str:
    '''아주 단순한 규칙 기반 요청 분류기(교육용).'''
    return "meeting" if ("회의" in user_request or "회의록" in user_request) else "generic"

def build_plan_rule_based(user_request: str):
    '''항상 유효한 실행 계획(dict)을 생성하는 Rule-based Planner.'''
    if classify_request(user_request) == "meeting":
        return {
            "steps": [
                {"tool": "make_summary_todos"},
                {"tool": "calculator", "expression": "30000000 * 0.15"},
                {"tool": "save_text", "filepath": "output/agent_result.txt"}
            ],
            "final_message": "회의록 처리를 완료했습니다."
        }
    return {
        "steps": [{"tool": "save_text", "filepath": "output/agent_result.txt"}],
        "final_message": "요청을 저장했습니다."
    }



## 4) Executor: 계획(steps) 실행

Executor는 Planner가 만든 계획을 **순서대로 실행**합니다.

- `make_summary_todos` → LLM Reasoner 호출(실패 시 fallback 포함)
- `calculator` → 수식 계산
- `save_text` → 결과를 파일로 저장


In [ ]:

def run_plan(plan, meeting_text):
    '''계획 실행기(Executor).
    - plan["steps"]를 순차 실행
    - 결과를 output/agent_result.txt로 저장
    '''
    summary, todos, calc_result = "", "", ""

    for step in plan["steps"]:
        if step["tool"] == "make_summary_todos":
            summary, todos = llm_make_summary_todos(meeting_text)

        elif step["tool"] == "calculator":
            calc_result = tool_calculator(step["expression"])

        elif step["tool"] == "save_text":
            content = f"""{summary}

{todos}

[계산] 3천만 원의 15% = {calc_result} 원
"""
            tool_save_text(step["filepath"], content)

        else:
            raise ValueError(f"Unknown tool: {step['tool']}")

    return plan["final_message"]



## 5) 데모 실행

실행 후 `output/agent_result.txt` 파일 내용을 확인하세요.


In [ ]:

USER_REQUEST = "회의록을 요약하고 후속 작업을 정리해줘."

MEETING_TEXT = """
- 마케팅 캠페인 일정 논의
- 예산은 2분기 내 집행
- SNS 홍보 강화 필요
- 다음 회의에서 세부 실행안 확정
"""

plan = build_plan_rule_based(USER_REQUEST)
print("PLAN:", json.dumps(plan, ensure_ascii=False, indent=2))

msg = run_plan(plan, MEETING_TEXT)
print("FINAL:", msg)

with open("output/agent_result.txt", "r", encoding="utf-8") as f:
    print("\n--- 저장된 파일 ---")
    print(f.read())



## 추가 학습 아이디어(교재 확장 포인트)

1. **분류기 고도화**: 규칙 기반 → 간단한 ML 분류기 → LLM 분류기로 비교  
2. **도구 확장**: 날씨/뉴스 API, SQLite DB 조회 등(9주차 Tools 연동과 연결)  
3. **RAG 연결**: 길어진 회의록에서 핵심 구간만 검색 후 요약(10주차 RAG와 연결)  
4. **품질 평가**: 요약 품질/To-do 적합성/재현성(Determinism) 측정  

---
